In [2]:
from joblib import load, dump
import numpy as np
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from scikeras.wrappers import KerasClassifier
import optuna


combined_data = load('../../DataDumps/3classfft.joblib')
combined_labels = load('../../DataDumps/3class_labels.joblib')

X_train_val, X_test, y_train_val, y_test = train_test_split(combined_data, combined_labels, test_size=0.2, random_state=42)
input_sh = X_train_val.shape

def create_model(optimizer, num_lstm, lstm_1, dropout_1, lstm_2 = 0, drop_2 = 0, lstm_3 = 0, drop_3 = 0):
    model = Sequential()
    model.add(LSTM(units=lstm_1, input_shape=(input_sh[1], input_sh[2]), return_sequences=True if num_lstm > 1 else False)) 
    model.add(Dropout(dropout_1))
    

    if num_lstm > 1:
        model.add(LSTM(units=lstm_2, input_shape=(input_sh[1], input_sh[2]), return_sequences=True if num_lstm == 3 else False)) 
        model.add(Dropout(drop_2))
    
    if num_lstm > 2:
        model.add(LSTM(units=lstm_3, input_shape=(input_sh[1], input_sh[2]), return_sequences= False)) 
        model.add(Dropout(drop_3))
    
    model.add(Dense(3, activation='softmax')) 

    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])   
    return model


# Optuna Objective Function
def objective(trial):
    lstm_2 = 0
    drop_2 = 0
    lstm_3 = 0
    drop_3 = 0
    num_lstm = trial.suggest_int('num_lstm_layers', 1, 3)
    lstm_1 = trial.suggest_int('lstm_units_1', 32, 128)
    dropout_1 = trial.suggest_float('dropout_1', 0.0, 0.5)
    optimizer = trial.suggest_categorical('optimizer', ['SGD', 'RMSprop', 'Adam'])

    if num_lstm > 1:
        lstm_2 = trial.suggest_int(f'lstm_units_2', 32, 128)
        drop_2 = trial.suggest_float(f'dropout_2', 0.0, 0.5)
    if num_lstm > 2:
        lstm_3 = trial.suggest_int(f'lstm_units_3', 32, 128)
        drop_3 = trial.suggest_float(f'dropout_3', 0.0, 0.5)


    # Assuming StratifiedKFold, customize if needed
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)  
    scores = []
    for train_index, test_index in skf.split(X_train_val, y_train_val):
        X_tr, X_val = X_train_val[train_index], X_train_val[test_index]
        y_tr, y_val = y_train_val[train_index], y_train_val[test_index]

        model = create_model(optimizer, num_lstm, lstm_1, dropout_1, lstm_2, drop_2, lstm_3, drop_3)
        model.fit(X_tr, y_tr,epochs = 20, verbose = 0) 
        score = model.evaluate(X_val, y_val, verbose = 0)
        scores.append(score[1])
    return np.array(scores).mean()

# Hyperparameter Optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

best_params = study.best_params
print("Best hyperparameters:", best_params)

[I 2024-04-24 01:15:19,016] A new study created in memory with name: no-name-e16c7edf-e2a6-4364-a5d3-6270977b9271
[I 2024-04-24 01:15:38,852] Trial 0 finished with value: 0.6008928775787353 and parameters: {'num_lstm_layers': 2, 'lstm_units_1': 82, 'dropout_1': 0.45402343195199063, 'optimizer': 'RMSprop', 'lstm_units_2': 39, 'dropout_2': 0.331961180864218}. Best is trial 0 with value: 0.6008928775787353.
[I 2024-04-24 01:15:58,760] Trial 1 finished with value: 0.5562500119209289 and parameters: {'num_lstm_layers': 2, 'lstm_units_1': 58, 'dropout_1': 0.21821507332346118, 'optimizer': 'RMSprop', 'lstm_units_2': 89, 'dropout_2': 0.4592522889047801}. Best is trial 0 with value: 0.6008928775787353.
[I 2024-04-24 01:16:25,678] Trial 2 finished with value: 0.5973214387893677 and parameters: {'num_lstm_layers': 3, 'lstm_units_1': 58, 'dropout_1': 0.23171983563400173, 'optimizer': 'RMSprop', 'lstm_units_2': 52, 'dropout_2': 0.12020686750201609, 'lstm_units_3': 110, 'dropout_3': 0.22828176644524

Best hyperparameters: {'num_lstm_layers': 3, 'lstm_units_1': 124, 'dropout_1': 0.10792333328469408, 'optimizer': 'RMSprop', 'lstm_units_2': 65, 'dropout_2': 0.49489384404853165, 'lstm_units_3': 40, 'dropout_3': 0.05397184830120454}


In [4]:
best_model = create_model("RMSprop", 3, 124, 0.10792333328469408,65,0.49489384404853165, 40, 0.05397184830120454)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.4, random_state=42)
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a ModelCheckpoint callback to save the best model weights
checkpoint = ModelCheckpoint(filepath='best_model_weights.h5', 
                              monitor='val_accuracy', 
                              save_best_only=True,
                              mode='max',
                              verbose=1)

# Train the model with the callback
history = best_model.fit(X_train, y_train,
                    epochs=400, 
                    validation_data=(X_val, y_val),
                    callbacks=[checkpoint])

# After training, load the best weights
best_model.load_weights('best_model_weights.h5')

# Evaluate the model using the best weights
loss, accuracy = best_model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')


Epoch 1/400
19/21 [==========================>...] - ETA: 0s - loss: 1.0586 - accuracy: 0.4539
Epoch 1: val_accuracy improved from -inf to 0.51786, saving model to best_model_weights.h5
21/21 [==============================] - 3s 45ms/step - loss: 1.0549 - accuracy: 0.4583 - val_loss: 1.0059 - val_accuracy: 0.5179
Epoch 2/400
20/21 [===========================>..] - ETA: 0s - loss: 0.9612 - accuracy: 0.5437
Epoch 2: val_accuracy improved from 0.51786 to 0.55580, saving model to best_model_weights.h5
21/21 [==============================] - 0s 14ms/step - loss: 0.9603 - accuracy: 0.5461 - val_loss: 0.9061 - val_accuracy: 0.5558
Epoch 3/400
21/21 [==============================] - ETA: 0s - loss: 0.8835 - accuracy: 0.5595
Epoch 3: val_accuracy improved from 0.55580 to 0.58036, saving model to best_model_weights.h5
21/21 [==============================] - 0s 14ms/step - loss: 0.8835 - accuracy: 0.5595 - val_loss: 0.8388 - val_accuracy: 0.5804
Epoch 4/400
21/21 [===========================